# Convolutional Neural Network in Keras

Bulding a Convolutional Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Load data

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [4]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_test = X_test.astype("float32")/255.

In [5]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### ReLUs

In [6]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

class ReLUs(Activation):
    
    def __init__(self, activation, **kwargs):
        super(ReLUs, self).__init__(activation, **kwargs)
        self.__name__ = 'relus'

def relus(Z):
    e_param = 10**-8
    pi = K.variable((3.14))
    m = e_param * (K.sigmoid(K.sin(Z)) - K.sigmoid(K.cos(Z)) * K.exp(K.sqrt(pi)))
    A = K.maximum(m, Z)
    return A

get_custom_objects().update({'relus': ReLUs(relus)})

#### Design Neural Network architecture

In [7]:
model = Sequential()

model.add(Conv2D(32, 7, padding = 'same', activation ='relus', input_shape = (28, 28, 1)))
model.add(Conv2D(32, 7, padding = 'same', activation ='relus'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, 3, padding = 'same', activation ='relus'))
model.add(Conv2D(64, 3, padding = 'same', activation ='relus'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.30))

model.add(Conv2D(128, 2, padding = 'same', activation ='relus'))
model.add(Conv2D(128, 2, padding = 'same', activation ='relus'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(512, activation = "relus"))
model.add(Dropout(0.50))
model.add(Dense(10, activation = "softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        1600      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        50208     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
__________

#### Callbacks

In [8]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-cnn-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')

tensorboard = TensorBoard("../logs/convnet-mnist-relus-III")

#### Configure model

In [9]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [10]:
model.fit(X_train, y_train, batch_size = 128, epochs = 30, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

Train on 54000 samples, validate on 6000 samples
Epoch 1/30
54000/54000 [==============================] - 460s 9ms/step - loss: 0.3422 - acc: 0.8860 - val_loss: 0.0561 - val_acc: 0.9837
Epoch 2/30
54000/54000 [==============================] - 453s 8ms/step - loss: 0.0848 - acc: 0.9743 - val_loss: 0.0326 - val_acc: 0.9902
Epoch 3/30
54000/54000 [==============================] - 475s 9ms/step - loss: 0.0623 - acc: 0.9814 - val_loss: 0.0316 - val_acc: 0.9908
Epoch 4/30
54000/54000 [==============================] - 479s 9ms/step - loss: 0.0518 - acc: 0.9842 - val_loss: 0.0268 - val_acc: 0.9928
Epoch 5/30
54000/54000 [==============================] - 478s 9ms/step - loss: 0.0453 - acc: 0.9866 - val_loss: 0.0270 - val_acc: 0.9915
Epoch 6/30
54000/54000 [==============================] - 497s 9ms/step - loss: 0.0396 - acc: 0.9879 - val_loss: 0.0227 - val_acc: 0.9923
Epoch 7/30
54000/54000 [==============================] - 509s 9ms/step - loss: 0.0386 - acc: 0.9882 - val_loss: 0.0242 - v

#### Test Predictions

In [11]:
saved_model = keras.models.load_model('model_output/weights-cnn-mnist.hdf5')
predictions = saved_model.predict_classes(X_test, verbose = 2)
print(predictions)

[7 2 1 ..., 4 5 6]


#### Test Final Accuracy

In [14]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

10000/10000 [==============================] - 30s 3ms/step
Final loss: 0.0183, final accuracy: 0.9946


In [13]:
#Final loss: 0.0204, final accuracy: 0.9940 (e = 0.001)